# Дипломный проект
# Тема

## Предсказать вероятность заражения компьютера с ОС Windows различными семействами вредоносных программ на основе различных свойств этого компьютера. Характеристики  и данные компьютера, содержащие эти свойства и заражения компьютеров, были получены путем объединения отчетов статических иследований Microsoft, Windows Defender ATP Research, Northeastern University College of Computer and Information Science, and Georgia Tech Institute for Information Security & Privacy.

## автор Колеганов Николай  

In [0]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
import gc
import scipy
import os
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
from sklearn import metrics

%matplotlib inline 
gc.enable()
plt.style.use('seaborn')
sns.set(font_scale=2)
pd.set_option('display.max_columns', 500)

## Подгружаем данные с диска

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!ls /content/drive/My\ Drive/ugroz

Просматриваем содержание

In [0]:
prosmotr = pd.read_csv('/content/drive/My Drive/ugroz/train.csv')

gc.enable()

In [0]:
prosmotr.head(10)


In [0]:
prosmotr.isnull().sum()

Как мы видим, довольно много категориальных признаков. В то же время, качество данных вызывает вопросы, довольно много пропущенных значений

In [0]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

gc.enable()

## Читаем и проверяем набор данных

In [0]:
tra = pd.read_csv('/content/drive/My Drive/ugroz/train.csv', dtype=dtypes, low_memory=True)
tra['MachineIdentifier'] = tra.index.astype('uint32')
gc.enable()

In [0]:
del prosmotr #удалим чтоб не занимал место
train = tra

In [0]:
print(train.info())

# Проверим соотношение распределение цели задачи
## 0 - не подвержен заражению
## 1 - подвержен заражению

In [0]:
train['HasDetections'].value_counts().plot.bar()
plt.title('HasDetections(target)')

# Проверяем данные

In [0]:
print('Размер датасета: ', train.shape)

In [0]:
train.head(5)

## есть ли пропуски в тренировочных данных и оцениваем их количество в %

In [0]:
total = train.isnull().sum().sort_values(ascending = False)
percent = (train.isnull().sum()/train.isnull().count()*100).sort_values(ascending = False)
missing_train_data  = pd.concat([total, percent], axis=1, keys=['Всего', 'Процентов'])

In [0]:
missing_train_data.head(50)


PuaMode, Census_ProcessorClass, DefaultBrowsersIdentifier, Census_IsFlightingInternal и Census_InternalBatteryType содержат более 70% нулевых данных.
Давайте проверим их распределение относительно цели.
Поскольку наборы данных велики, давайте сравним распределения, используя 10% train

In [0]:
train_small = train

# Посмотрим какие уникальные значения и их количество получилось

In [0]:
train_small.nunique()

# Смотрим зависимости признаков и графики

# PuaMode - Pua включен режим из сервиса

In [0]:
print(train_small['PuaMode'].dtypes)

In [0]:
sns.countplot(x='PuaMode', hue='HasDetections',data=train_small)
plt.show()


Там есть какая-то разница. Но образцов довольно мало, поэтому удалим эту функцию.

# Классификация процессоров на высокий / средний / низкий.
##  Census_ProcessorClass

In [0]:
print(train_small['Census_ProcessorClass'].dtypes)

In [0]:
sns.countplot(x='Census_ProcessorClass', hue='HasDetections',data=train_small)
plt.show()

«Census_ProcessorClassr» - «Количество логических ядер в процессоре».
Вы можете проверить, что чем больше логических ядер, тем вероятнее заражение вредоносными программами.
Эта функция может быть только хорошими функциями или компонентом для комбинаций с другими функциями.

# ID для браузера по умолчанию на машине
## DefaultBrosersIdentifier

In [0]:
print(train_small['DefaultBrowsersIdentifier'].dtypes)

In [0]:
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, 'DefaultBrowsersIdentifier'], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, 'DefaultBrowsersIdentifier'], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, 'DefaultBrowsersIdentifier'].hist(ax=ax[1])
train_small.loc[train['HasDetections'] == 1, 'DefaultBrowsersIdentifier'].hist(ax=ax[1])
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])

plt.show()


DefaultBrowsersIdentifier означает «идентификатор браузера машины по умолчанию».
Является ли эта функция значимой?

# Census_IsFlightingInternal - нет данных

In [0]:
print(train_small['Census_IsFlightingInternal'].dtypes)

In [0]:
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sns.kdeplot(train_small.loc[train['HasDetections'] == 0, 'Census_IsFlightingInternal'], ax=ax[0], label='NoDetection(0)')
sns.kdeplot(train_small.loc[train['HasDetections'] == 1, 'Census_IsFlightingInternal'], ax=ax[0], label='HasDetection(1)')

train_small.loc[train['HasDetections'] == 0, 'Census_IsFlightingInternal'].hist(ax=ax[1])
train_small.loc[train['HasDetections'] == 1, 'Census_IsFlightingInternal'].hist(ax=ax[1])
ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])

plt.show()

In [0]:
train_small.loc[train['HasDetections'] == 1, 'Census_IsFlightingInternal'].value_counts()

In [0]:
train_small.loc[train['HasDetections'] == 0, 'Census_IsFlightingInternal'].value_counts()


Как видите, значение Census_IsFlightingInternal почти равно 0,0. Просто удалим

# Census_InternalBatteryType - нет данных

In [0]:
print(train_small['Census_InternalBatteryType'].dtypes)

In [0]:
train_small['Census_InternalBatteryType'].value_counts()


Я думаю, что эта функция означает тип батареи каждой машины.
В наши дни большинство батарей - это литий-ионные аккумуляторы.
И сгруппировал их в группу литий и группу без-литиевая батарея

In [0]:
def group_battery(x):
    x = x.lower()
    if 'li' in x:
        return 1
    else:
        return 0
    
train_small['Census_InternalBatteryType'] = train_small['Census_InternalBatteryType'].apply(group_battery)

In [0]:
sns.countplot(x='Census_InternalBatteryType', hue='HasDetections',data=train_small)
plt.show()


Разница довольно маленькая.
Возник вопрос...."некоторые вредоносные программы распознают и выбирают машину в зависимости от типа батареи?"

Аккумулятор очень важная часть для жизни машины. Я думаю, что вредоносные программы будут сосредоточены на других аппаратных и программных частях машины.
Поэтому его можно удалить.

In [0]:
null_cols_to_remove = ['DefaultBrowsersIdentifier', 'PuaMode',
                       'Census_IsFlightingInternal', 'Census_InternalBatteryType']

train.drop(null_cols_to_remove, axis=1, inplace=True)

## Получившиеся предварительные данные

In [0]:
print(train.shape)

In [0]:
train.head(25)

In [0]:
train.isnull().sum()

## Как видим есть еще пропуски, но уже меньше. Проанализируем каждый парамметр на то как он влияет на определение подвержен АРМ заражению или нет

# Проведем исследовательский анализ данных дальше

## Категориальные особенности

In [0]:
categorical_features = [
        'ProductName',                                          
        'EngineVersion',                                        
        'AppVersion',                                           
        'AvSigVersion',                                         
        'Platform',                                             
        'Processor',                                            
        'OsVer',                                                
        'OsPlatformSubRelease',                                 
        'OsBuildLab',                                           
        'SkuEdition',                                           
        'SmartScreen',                                          
        'Census_MDC2FormFactor',                                
        'Census_DeviceFamily',                                  
        'Census_PrimaryDiskTypeName',                           
        'Census_ChassisTypeName',                               
        'Census_PowerPlatformRoleName',                         
        'Census_OSVersion',                                     
        'Census_OSArchitecture',                                
        'Census_OSBranch',                                      
        'Census_OSEdition',                                     
        'Census_OSSkuName',                                     
        'Census_OSInstallTypeName',                             
        'Census_OSWUAutoUpdateOptionsName',                     
        'Census_GenuineStateName',                              
        'Census_ActivationChannel',                             
        'Census_FlightRing',                                    
]

In [0]:
total = train.isnull().sum().sort_values(ascending = False)
percent = (train.isnull().sum()/train.isnull().count()*100).sort_values(ascending = False)
missing_train_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [0]:
missing_train_data.head(50)

## ProductName - информация о состоянии Защитника, например win8defender

In [0]:
def plot_category_percent_of_target(col):
    fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    cat_percent = train_small[[col, 'HasDetections']].groupby(col, as_index=False).mean()
    cat_size = train_small[col].value_counts().reset_index(drop=False)
    cat_size.columns = [col, 'count']
    cat_percent = cat_percent.merge(cat_size, on=col, how='left')
    cat_percent['HasDetections'] = cat_percent['HasDetections'].fillna(0)
    cat_percent = cat_percent.sort_values(by='count', ascending=False)[:20]
    sns.barplot(ax=ax, x='HasDetections', y=col, data=cat_percent, order=cat_percent[col])

    for i, p in enumerate(ax.patches):
        ax.annotate('{}'.format(cat_percent['count'].values[i]), (p.get_width(), p.get_y()+0.5), fontsize=20)

    plt.xlabel('% of HasDetections(target)')
    plt.ylabel(col)
    plt.show()

In [0]:
col = categorical_features[0]
plot_category_percent_of_target(col)

## EngineVersion - информация о состоянии Защитника, например, 1.1.12603.0

In [0]:
col = categorical_features[1]
plot_category_percent_of_target(col)

## AppVersion - информация о состоянии Защитника, например, 4.9.10586.0

In [0]:
col = categorical_features[2]
plot_category_percent_of_target(col)

## AvSigVersion - информация о состоянии Защитника, например, 1.217.1014.0

In [0]:
col = categorical_features[3]
plot_category_percent_of_target(col)

## Платформа - вычисляет имя платформы (связанных свойств ОС и свойств процессора)

In [0]:
col = categorical_features[4]
plot_category_percent_of_target(col)

## Процессор - это архитектура процесса установленной операционной системы.

In [0]:
col = categorical_features[5]
plot_category_percent_of_target(col)

## Census_OSVersion - Пример числовой версии ОС - 10.0.10130.0

In [0]:
col = categorical_features[6]
plot_category_percent_of_target(col)


## Возвращает дополнительную версию платформы ОС (Windows Vista, Windows 7, Windows 8, TH1, TH2)

In [0]:
col = categorical_features[7]
plot_category_percent_of_target(col)

## Построить лабораторию, которая генерировала текущую ОС. Пример: 9600.17630.amd64fre.winblue_r7.150109-2022

In [0]:
col = categorical_features[8]
plot_category_percent_of_target(col)

## SkuEdition- Цель этой функции - использовать тип продукта, определенный в MSDN, для сопоставления с именем «SKU-Edition», которое полезно при составлении отчетов о населении. Допустимый тип продукта определен в% sdxroot% \ data \ windowseditions.xml. Этот API используется с Vista и Server 2008, поэтому существует много типов продуктов, которые не применимы к Windows 10. «SKU-Edition» - это строковое значение, которое входит в один из трех классов результатов. Дизайн должен передать каждый класс.

In [0]:
col = categorical_features[9]
plot_category_percent_of_target(col)

## SmartScreen- Это активированное строковое значение SmartScreen из реестра. Это получается путем проверки по порядку: HKLM \ SOFTWARE \ Policies \ Microsoft \ Windows \ System \ SmartScreenEnabled и HKLM \ SOFTWARE \ Microsoft \ Windows \ CurrentVersion \ Explorer \ SmartScreenEnabled.

In [0]:
col = categorical_features[10]
plot_category_percent_of_target(col)

## Census_MDC2FormFactor- Группировка на основе комбинации характеристик оборудования на уровне переписи устройства. Логика, используемая для определения форм-фактора, основана на деловых и отраслевых стандартах и соответствует тому, как люди думают об их устройстве. (Примеры: смартфон, маленький планшет, все в одном, кабриолет ...)

In [0]:
col = categorical_features[11]
plot_category_percent_of_target(col)

##  Census_DeviceFamily- AKA DeviceClass. Указывает тип устройства, для которого предназначена редакция ОС. 

In [0]:
col = categorical_features[12]
plot_category_percent_of_target(col)

## Census_PrimaryDiskTotalCapacity - Используемые носители  на основном диске машины в МБ

In [0]:
col = categorical_features[13]
plot_category_percent_of_target(col)

## Census_ChassisTypeName - Извлекает числовое представление типа шасси машины

In [0]:
col = categorical_features[14]
plot_category_percent_of_target(col)

## Census_PowerPlatformRoleName- Указывает предпочитаемый OEM профиль управления питанием. Это значение помогает определить базовый форм-фактор устройства

In [0]:
col = categorical_features[15]
plot_category_percent_of_target(col)

## Census_OSVersion - Пример числовой версии ОС 

In [0]:
col = categorical_features[16]
plot_category_percent_of_target(col)

## Census_OSArchitecture- Архитектура, на которой основана ОС

In [0]:
col = categorical_features[17]
plot_category_percent_of_target(col)

## Версия ОС извлечена из OsVersionFull

In [0]:
col = categorical_features[18]
plot_category_percent_of_target(col)

## Census_OSEdition- Выпуск текущей ОС
 Получается из реестра HKLM \ Software \ Microsoft \ Windows NT \ CurrentVersion @ EditionID в реестре.

In [0]:
col = categorical_features[19]
plot_category_percent_of_target(col)

## Census_OSSkuName - дружественное имя для выпуска ОС 

In [0]:
col = categorical_features[20]
plot_category_percent_of_target(col)

## Census_PrimaryDiskTypeName - Дружественное имя основного типа диска - HDD или SSD

In [0]:
col = categorical_features[21]
plot_category_percent_of_target(col)

## Census_OSWUAutoUpdateOptionsName - Дружественное имя настроек автообновления 

In [0]:
col = categorical_features[22]
plot_category_percent_of_target(col)

## Census_GenuineStateName- Дружественное имя OSGenuineStateID. 


In [0]:
col = categorical_features[23]
plot_category_percent_of_target(col)

## Census_ActivationChannel - Розничный лицензионный ключ или корпоративный лицензионный ключ для машины

In [0]:
col = categorical_features[24]
plot_category_percent_of_target(col)

## Census_IsFlightingInternal - нет данных

In [0]:
col = categorical_features[25]
plot_category_percent_of_target(col)

# Числовые особенности

In [0]:
numeric_features = [
        'IsBeta',                                               
        'RtpStateBitfield',                                     
        'IsSxsPassiveMode',                                     
        'DefaultBrowsersIdentifier',                            
        'AVProductStatesIdentifier',                            
        'AVProductsInstalled',                                  
        'AVProductsEnabled',                                    
        'HasTpm',                                               
        'CountryIdentifier',                                    
        'CityIdentifier',                                       
        'OrganizationIdentifier',                               
        'GeoNameIdentifier',                                    
        'LocaleEnglishNameIdentifier',                          
        'OsBuild',                                              
        'OsSuite',                                              
        'IsProtected',                                          
        'AutoSampleOptIn',                                      
        'SMode',                                                
        'IeVerIdentifier',                                      
        'Firewall',                                             
        'UacLuaenable',                                         
        'Census_OEMNameIdentifier',                             
        'Census_OEMModelIdentifier',                            
        'Census_ProcessorCoreCount',                            
        'Census_ProcessorManufacturerIdentifier',               
        'Census_ProcessorModelIdentifier',                      
        'Census_PrimaryDiskTotalCapacity',                      
        'Census_SystemVolumeTotalCapacity',                     
        'Census_HasOpticalDiskDrive',                           
        'Census_TotalPhysicalRAM',                              
        'Census_InternalPrimaryDiagonalDisplaySizeInInches',    
        'Census_InternalPrimaryDisplayResolutionHorizontal',    
        'Census_InternalPrimaryDisplayResolutionVertical',      
        'Census_InternalBatteryNumberOfCharges',                
        'Census_OSBuildNumber',                                 
        'Census_OSBuildRevision',                               
        'Census_OSInstallLanguageIdentifier',                   
        'Census_OSUILocaleIdentifier',                          
        'Census_IsPortableOperatingSystem',                     
        'Census_IsFlightsDisabled',                             
        'Census_ThresholdOptIn',                                
        'Census_FirmwareManufacturerIdentifier',                
        'Census_FirmwareVersionIdentifier',                     
        'Census_IsSecureBootEnabled',                           
        'Census_IsWIMBootEnabled',                              
        'Census_IsVirtualDevice',                               
        'Census_IsTouchEnabled',                                
        'Census_IsPenCapable',                                  
        'Census_IsAlwaysOnAlwaysConnectedCapable',              
        'Wdft_IsGamer',                                         
        'Wdft_RegionIdentifier',                                
]

## Информация о состоянии защитника, например, ложь

In [0]:
def plot_category_percent_of_target_for_numeric(col):
    fig, ax = plt.subplots(1, 2, figsize=(18, 8))
    cat_percent = train_small[[col, 'HasDetections']].groupby(col, as_index=False).mean()
    cat_size = train_small[col].value_counts().reset_index(drop=False)
    cat_size.columns = [col, 'count']
    cat_percent = cat_percent.merge(cat_size, on=col, how='left')
    cat_percent['HasDetections'] = cat_percent['HasDetections'].fillna(0)
    cat_percent = cat_percent.sort_values(by='count', ascending=False)[:20]
    cat_percent[col] = cat_percent[col].astype('category')
    sns.barplot(ax=ax[0], x='HasDetections', y=col, data=cat_percent,  order=cat_percent[col])

    for i, p in enumerate(ax[0].patches):
        ax[0].annotate('{}'.format(cat_percent['count'].values[i]), (p.get_width(), p.get_y()+0.5), fontsize=20)

    ax[0].set_title('Barplot sorted by count', fontsize=20)

    sns.barplot(ax=ax[1], x='HasDetections', y=col, data=cat_percent)
    for i, p in enumerate(ax[0].patches):
        ax[1].annotate('{}'.format(cat_percent['count'].sort_index().values[i]), (0, p.get_y()+0.6), fontsize=20)
    ax[1].set_title('Barplot sorted by index', fontsize=20)

    plt.xlabel('% of HasDetections(target)')
    plt.ylabel(col)
    plt.subplots_adjust(wspace=0.5, hspace=0)
    plt.show()

def plot_kde_hist_for_numeric(col):
    fig, ax = plt.subplots(1, 2, figsize=(14, 8))
    sns.kdeplot(train_small.loc[train['HasDetections'] == 0, col], ax=ax[0], label='NoDetection(0)')
    sns.kdeplot(train_small.loc[train['HasDetections'] == 1, col], ax=ax[0], label='HasDetection(1)')

    train_small.loc[train['HasDetections'] == 0, col].hist(ax=ax[1], bins=100)
    train_small.loc[train['HasDetections'] == 1, col].hist(ax=ax[1], bins=100)

    plt.suptitle(col, fontsize=30)
    ax[0].set_yscale('log')
    ax[0].set_title('KDE plot')
    
    ax[1].set_title('Histogram')
    ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])
    ax[1].set_yscale('log')
    plt.show()

In [0]:
col = numeric_features[0]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## RtpStateBitfield

In [0]:
col = numeric_features[1]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

##  IsSxsPassiveMode

In [0]:
col = numeric_features[2]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## IID для конкретной конфигурации антивирусного программного обеспечения пользователя

In [0]:
col = numeric_features[4]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## AVProductsInstalled

In [0]:
col = numeric_features[5]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## AVProductsEnabled

In [0]:
col = numeric_features[6]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Правда, если машина имеет tpm

In [0]:
col = numeric_features[7]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Идентификатор страны, в которой находится машина

In [0]:
col = numeric_features[8]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## ID для города, в котором находится машина

In [0]:
col = numeric_features[9]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Идентификатор организации, к которой принадлежит машина, идентификатор 
идентификатор организации сопоставлен как с конкретными компаниями, так и с широкими отраслями

In [0]:
col = numeric_features[10]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Идентификатор географического региона, в котором находится машина

In [0]:
col = numeric_features[11]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Locale ID пользователя

In [0]:
col = numeric_features[12]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Сборка операционной системы

In [0]:
col = numeric_features[13]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Маска набора продуктов для операционной системы.

In [0]:
col = numeric_features[14]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Это вычисляемое поле, полученное из поля AV Products отчета Spynet. Парамметра IsProtected

#True, если на этом компьютере работает хотя бы один активный и современный антивирусный продукт
#FALSE, если на этом аппарате нет активного продукта AV, или если AV активен, но не получает последние обновления. с. null, если в отчете нет антивирусных продуктов. Возвращает: защищена ли машина.

In [0]:
col = numeric_features[15]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Значение SubmitSamplesConsent, переданное службой и доступное на CAMP 9+

In [0]:
col = numeric_features[16]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## SMode - Это поле имеет значение true, если известно, что устройство находится в «режиме S», например, в режиме Windows 10 S, где могут быть установлены только приложения Microsoft Store.

In [0]:
col = numeric_features[17]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## IeVerIdentifier

In [0]:
col = numeric_features[18]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Firewall

In [0]:
col = numeric_features[19]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Атрибут сообщает, отключен ли тип пользователя «Администратор в режиме одобрения администратором» или включен в UAC. 
Сообщаемое значение получается путем чтения regkey HKLM \ SOFTWARE \ Microsoft \ Windows \ CurrentVersion \ Policies \ System \ EnableLUA.

In [0]:
col = numeric_features[20]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Census_OEMNameIdentifier

In [0]:
col = numeric_features[21]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Census_OEMModelIdentifier

In [0]:
col = numeric_features[22]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Количество логических ядер в процессоре

In [0]:
col = numeric_features[23]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Census_ProcessorManufacturerIdentifier

In [0]:
col = numeric_features[24]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Census_ProcessorModelIdentifier

In [0]:
col = numeric_features[25]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Объем дискового пространства на основном диске машины в МБ

In [0]:
col = numeric_features[26]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Размер раздела, на котором установлен системный том, в МБ

In [0]:
col = numeric_features[27]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## На  машине установлен дисковод оптических дисков (CD / DVD)

In [0]:
col = numeric_features[28]

plot_category_percent_of_target_for_numeric(col)

## Физическая память RAM в МБ

In [0]:
col = numeric_features[29]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Физическая длина диагонали в дюймах основного дисплея

In [0]:
col = numeric_features[30]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Получаемое количество пикселей в горизонтальном направлении внутреннего дисплея.

In [0]:
col = numeric_features[31]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Получает количество пикселей в вертикальном направлении внутреннего дисплея

In [0]:
col = numeric_features[32]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Номер сборки ОС, извлеченный из OsVersionFull.

In [0]:
col = numeric_features[34]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Census_InternalBatteryNumberOfCharges 

In [0]:
col = numeric_features[36]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Census_OSInstallLanguageIdentifier 

In [0]:
col = numeric_features[37]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Указывает, загружена ли ОС и работает ли она через Windows-To-Go на USB-накопителе

In [0]:
col = numeric_features[38]

plot_category_percent_of_target_for_numeric(col)

## Указывает, участвует ли машина в полете.

In [0]:
col = numeric_features[39]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Census_ThresholdOptIn

In [0]:
col = numeric_features[40]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Census_FirmwareManufacturerIdentifier

In [0]:
col = numeric_features[41]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Census_FirmwareVersionIdentifier

In [0]:
col = numeric_features[42]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Census_IsSecureBootEnabled - Указывает, включен ли режим безопасной загрузки.

In [0]:
col = numeric_features[43]

plot_category_percent_of_target_for_numeric(col)

## Census_IsWIMBootEnabled 

In [0]:
col = numeric_features[44]

plot_category_percent_of_target_for_numeric(col)

## Идентифицирует виртуальную машину 

In [0]:
col = numeric_features[45]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Census_IsTouchEnabled - Это сенсорное устройство

In [0]:
col = numeric_features[46]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Census_IsPenCapable - Способно ли устройство к ручному вводу?

In [0]:
col = numeric_features[47]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Census_IsAlwaysOnAlwaysConnectedCapable - Получает информацию о том, позволяет ли аккумулятор постоянно подключать устройство.

In [0]:
col = numeric_features[48]

plot_category_percent_of_target_for_numeric(col)

## Указывает, является ли устройство игровым устройством или нет на основе его аппаратной комбинации.

In [0]:
col = numeric_features[49]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

## Wdft_RegionIdentifier 

In [0]:
col = numeric_features[50]

plot_kde_hist_for_numeric(col)
plot_category_percent_of_target_for_numeric(col)

# Корреляция

In [0]:
gc.enable()
corr = train_small.corr()['HasDetections']

In [0]:
abs(corr).sort_values(ascending=False)

In [0]:
def corr_heatmap(cols):
    correlations = train_small[cols+['HasDetections']].corr()
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    fig, ax = plt.subplots(figsize=(15, 15))
    sns.heatmap(correlations, cmap=cmap, vmax=1.0, center=0, fmt='.2f',
                square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .75})
    plt.show()

## Делаем кореляцию признаков

In [0]:
numeric_features

In [0]:
corr_heatmap(numeric_features[:3])

In [0]:
corr_heatmap(numeric_features[4:10])

In [0]:
corr_heatmap(numeric_features[10:20])

In [0]:
corr_heatmap(numeric_features[20:30])

In [0]:
corr_heatmap(numeric_features[30:40])

In [0]:
corr_heatmap(numeric_features[40:])

In [0]:
corr = train_small.corr()

In [0]:
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))
upper.head()

In [0]:
threshold = 0.3
for i, df in (upper.iterrows()):
    for ele in df[df.abs() > threshold].items():
        if ele[0] == i:
            break
        else:
            plt.figure(figsize=(7, 7))
            sns.lmplot(x=i, y=ele[0], data=train_small[:100000], hue='HasDetections', palette='Set1', scatter_kws={'alpha':0.3})
            plt.show()
            print('{:50}, {:50} : {}'.format(i, ele[0], ele[1]))

## Многие признаки являются категориальными, и пары, которые имеют высокие корреляции, также состоят из категориальных признаков. Я думаю, что ключевым моментом является создание некоторых функций, которые имеют категории с высокой вероятностью заражения вредоносными программами, а некоторые функции являются избыточными.

In [0]:
train_small.head(3)
  
gc.collect()    

# Удаляем на мой взгляд не нужные параметры для определения вероятности заражения, хотя в некоторых моментах они нужны, например для определения глобальной статистики заражения по странам, городам, организациям, группам пользователей

In [0]:
train = train_small
print(type(train))

In [0]:
null_remove = ['LocaleEnglishNameIdentifier', 'GeoNameIdentifier', 'OrganizationIdentifier',
              'MachineIdentifier']

train.drop(null_remove, axis=1, inplace=True)

In [0]:
!ls

In [0]:
!mv dat.csv /content/drive/My\ Drive/ugroz

In [0]:
train.info()

## Кодируем получившиеся данные для обучения

In [0]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
import gc
import scipy
import os
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
from sklearn import metrics

%matplotlib inline 
gc.enable()
plt.style.use('seaborn')
sns.set(font_scale=2)
pd.set_option('display.max_columns', 500)

## Данные которые будем кодировать разделил на FE и OHE

In [0]:
FE = ['EngineVersion','AppVersion','AvSigVersion','Census_OSVersion']


OHE = [ 'RtpStateBitfield','IsSxsPassiveMode',
        'AVProductStatesIdentifier','AVProductsInstalled', 'AVProductsEnabled',
        'Processor', 'OsBuild', 'OsSuite',
        'SmartScreen','Census_MDC2FormFactor',
        'Census_OEMNameIdentifier', 
        'Census_ProcessorCoreCount',
        'Census_ProcessorModelIdentifier', 
        'Census_PrimaryDiskTotalCapacity', 'Census_PrimaryDiskTypeName',
        'Census_HasOpticalDiskDrive',
        'Census_TotalPhysicalRAM', 'Census_ChassisTypeName',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches',
        'Census_InternalPrimaryDisplayResolutionHorizontal',
        'Census_InternalPrimaryDisplayResolutionVertical',
        'Census_PowerPlatformRoleName',
        'Census_InternalBatteryNumberOfCharges',
        'Census_OSEdition', 'Census_OSInstallLanguageIdentifier',
        'Census_GenuineStateName','Census_ActivationChannel',
        'Census_FirmwareManufacturerIdentifier',
        'Census_IsTouchEnabled', 'Census_IsPenCapable','CityIdentifier', 
        'CountryIdentifier','Census_IsAlwaysOnAlwaysConnectedCapable',
        'Wdft_IsGamer','Wdft_RegionIdentifier']

In [0]:
# Загрузка всех как категориальных признаков
dtypes = {}
for x in FE+OHE: 
  dtypes[x] = 'category'
  dtypes['Unnamed: 0'] = 'str'
  dtypes['HasDetections'] = 'int8'

# Загрузка данных
df_train = pd.read_csv('/content/drive/My Drive/ugroz/dat.csv', usecols=dtypes.keys(), dtype=dtypes)
print ('Загруженно',len(df_train),'строк набора предобработанных данных!')

# Параметр используемых для обучения и тренировки
sm = 2900000
df_train = df_train.sample(sm)
print ('Использовать буду',sm,' для тренировочных и проверочных данных')
x=gc.collect()

In [0]:
import math
# Проверка на NaN
def nan_check(x):
    if isinstance(x,float):
        if math.isnan(x):
            return True
    return False

# Частотное кодирование
def encode_FE(df,col,verbose=1):
    d = df[col].value_counts(dropna=False)
    n = col+"_FE"
    df[n] = df[col].map(d)/d.max()
    if verbose==1:
        print('FE encoded',col)
    return [n]

# ONE-HOT-ENCODE ALL CATEGORY VALUES THAT COMPRISE MORE THAN
# "FILTER" PERCENT OF TOTAL DATA AND HAS SIGNIFICANCE GREATER THAN "ZVALUE"
def encode_OHE(df, col, filter, zvalue, tar='HasDetections', m=0.5, verbose=1):
    cv = df[col].value_counts(dropna=False)
    cvd = cv.to_dict()
    vals = len(cv)
    th = filter * len(df)
    sd = zvalue * 0.5/ math.sqrt(th)
    print('Отклонение', sd)
    n = []; ct = 0; d = {}
    for x in cv.index:
        try:
            if cv[x]<th: break
            sd = zvalue * 0.5/ math.sqrt(cv[x])
        except:
            if cvd[x]<th: break
            sd = zvalue * 0.5/ math.sqrt(cvd[x])
        if nan_check(x): r = df[df[col].isna()][tar].mean()
        else: r = df[df[col]==x][tar].mean()
        if abs(r-m)>sd:
            nm = col+'_BE_'+str(x)
            if nan_check(x): df[nm] = (df[col].isna()).astype('int8')
            else: df[nm] = (df[col]==x).astype('int8')
            n.append(nm)
            d[x] = 1
        ct += 1
        if (ct+1)>=vals: break
    if verbose==1:
        print('OHE encoded',col,'- Created',len(d),'booleans')
    return [n,d]

# ONE-HOT-ENCODING
def encode_OHE_test(df,col,dt):
    n = []
    for x in dt: 
        n += encode_BE(df,col,x)
    return n

# BOOLEAN ENCODING
def encode_BE(df,col,val):
    n = col+"_BE_"+str(val)
    if nan_check(val):
        df[n] = df[col].isna()
    else:
        df[n] = df[col]==val
    df[n] = df[n].astype('int8')
    return [n]

In [0]:
from sklearn.preprocessing import LabelEncoder
LabelEncoder()
cols = []; dd = []

# ENCODE NEW
for x in FE:
    cols += encode_FE(df_train,x)
for x in OHE:
    tmp = encode_OHE(df_train,x,0.005,5)
    cols += tmp[0]; dd.append(tmp[1])
print('Encoded',len(cols),'new variables')

# REMOVE OLD
for x in FE+OHE:
    del df_train[x]
print('Removed original',len(FE+OHE),'variables')
x = gc.collect()

## Создаем callback с лучшим результатом

In [0]:
from keras import callbacks
from sklearn.metrics import roc_auc_score

class printAUC(callbacks.Callback):
    def __init__(self, X_train, Y_train):
        super(printAUC, self).__init__()
        self.bestAUC = 0
        self.X_train = X_train
        self.y_train = Y_train
        
    def on_epoch_end(self, epoch, logs={}):
        pred = self.model.predict(np.array(self.X_train))
        auc = roc_auc_score(self.y_train, pred)
        print("Тренировочная AUC: " + str(auc))
        pred = self.model.predict(self.validation_data[0])
        auc = roc_auc_score(self.validation_data[1], pred)
        print ("Проверочная AUC: " + str(auc))
        if (self.bestAUC < auc) :
            self.bestAUC = auc
            self.model.save("bestNet.h5", overwrite=True)
            gc.enable()
        return

In [0]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout,LSTM, GRU, BatchNormalization, Embedding,Flatten, SpatialDropout1D ,MaxPooling1D, Conv1D, GlobalMaxPooling1D, Activation
from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam, Adamax, Adagrad



## Проверил содержание данных на наличие пропусков

In [0]:
df_train.isnull().sum()

## Разбиваем данные для обучения и проверки

In [0]:
X_train, X_val, Y_train, Y_val = train_test_split(
    df_train[cols], df_train['HasDetections'], test_size = 0.3)


In [0]:
Y_train.shape

In [0]:
X_train.shape


In [0]:
X_val.shape

In [0]:
Y_val.shape

## Сохраняем получившиеся данные для обчения и проверки на диск

In [0]:
X_train.to_csv('X_train4.csv', header=False, index=False)
!mv X_train4.csv /content/drive/My\ Drive/ugroz/train_dan

In [0]:
X_val.to_csv('X_val4.csv',header=False, index=False)
!mv X_val4.csv /content/drive/My\ Drive/ugroz/train_dan

In [0]:
Y_train.to_csv('Y_train4.csv', header=False, index=False)
!mv Y_train4.csv /content/drive/My\ Drive/ugroz/train_dan

In [0]:
Y_val.to_csv('Y_val4.csv', header=False, index=False)
!mv Y_val4.csv /content/drive/My\ Drive/ugroz/train_dan

# Обработка и анализ данных завершили, переходим к построению сетей

## Dense с tanh

In [0]:
import keras

In [0]:
model = Sequential()
model.add(Dense(100,input_dim=len(cols), kernel_initializer='normal', activation='tanh'))
model.add(Dense(160,kernel_initializer='normal', activation='tanh'))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [0]:
model.compile(optimizer=Adam(lr=0.01), loss="binary_crossentropy", metrics=["accuracy"])
annealer = LearningRateScheduler(lambda x: 1e-2 * 0.65 ** x)

In [0]:
history=model.fit(X_train,Y_train, batch_size=128, epochs = 15, validation_data = (X_val,Y_val), 
                  verbose=1, callbacks=[annealer, printAUC(X_train, Y_train)])

In [0]:
plt.plot(history.history['acc'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_acc'], 
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

gc.enable()

In [0]:
!ls

In [0]:
!mv bestNet.h5 /content/drive/My\ Drive/ugroz/DENSE/tanh

## Dense с relu

In [0]:
model1 = Sequential()
model1.add(Dense(800,input_dim=len(cols)))
model1.add(Dropout(0.25))
model1.add(BatchNormalization()
model1.add(Activation('relu'))
model1.add(Dense(260))
model1.add(Dense(260))
model1.add(Dropout(0.3))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Dense(180))
model1.add(Dense(180))
model1.add(Dropout(0.4))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.summary()

In [0]:
model1.compile(optimizer=Adam(lr=0.01), loss="binary_crossentropy", metrics=["accuracy"])
annealer = LearningRateScheduler(lambda x: 1e-2 * 0.65 ** x)

In [0]:
history1=model1.fit(X_train,Y_train, batch_size=128, epochs = 15, validation_data = (X_val,Y_val), 
                  verbose=1, callbacks=[annealer, printAUC(X_train, Y_train)])

In [0]:
plt.plot(history1.history['acc'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(history1.history['val_acc'], 
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

gc.enable()

In [0]:
!ls

In [0]:
!mv bestNet.h5 /content/drive/My\ Drive/ugroz/DENSE/relu

# Другая сетка c LSTM

In [0]:
model2 = Sequential()
model2.add(Embedding(len(cols), 64))
model2.add(LSTM(64, return_sequences=True))
model2.add(BatchNormalization())
model2.add(LSTM(32))
model2.add(BatchNormalization())
model2.add(Dense(1, activation='sigmoid'))
model2.summary()

In [0]:
model2.compile(optimizer='adagrad', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])


In [0]:
history2=model2.fit(X_train,Y_train, batch_size=1024, epochs = 5, validation_data = (X_val,Y_val), 
                    verbose=1) 
                  

In [0]:
plt.plot(history2.history['acc'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(history2.history['val_acc'], 
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

In [0]:
model2.save("bestNetLSTM.h5", overwrite=True)

In [0]:
!ls 

In [0]:
!mv bestNetLSTM.h5 /content/drive/My\ Drive/ugroz/LSTM

# Сетка со слоем GRU

In [0]:
model3 = Sequential()
model3.add(Embedding(len(cols), 32))
model3.add(GRU(32))
model3.add(Dense(64))
model3.add(Dropout(0.25))
model3.add(BatchNormalization())
model3.add(Dense(1, activation='sigmoid'))
model3.summary()

In [0]:
model3.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])


In [0]:
history3=model3.fit(X_train,Y_train, batch_size=1024, epochs = 15, validation_data = (X_val,Y_val), 
                    verbose=1)


In [0]:
plt.plot(history3.history['acc'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(history3.history['val_acc'], 
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

In [0]:
model3.save("bestNetGRU.h5", overwrite=True)

In [0]:
!mv bestNetGRU.h5 /content/drive/My\ Drive/ugroz/GRU

# Одномерная свёртка

In [0]:
model4 = Sequential()
model4.add(Embedding(len(cols), 128))
model4.add(Conv1D(128, 5))
model4.add(Activation('relu'))
model4.add(Conv1D(128, 5))
model4.add(Activation('relu'))
model4.add(BatchNormalization())
model4.add(GlobalMaxPooling1D())
model4.add(Dense(64))
model4.add(Activation('relu'))
model4.add(Dense(1, activation='sigmoid'))
model4.summary()

In [0]:
model4.compile(optimizer=Adam(lr=0.01), loss="binary_crossentropy", metrics=["accuracy"])


In [0]:
history4=model4.fit(X_train,Y_train, batch_size=128, epochs = 15, validation_data = (X_val,Y_val), 
                  verbose=1)

In [0]:
plt.plot(history4.history['acc'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(history4.history['val_acc'], 
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

gc.enable()

In [0]:
model4.save("bestNet1D.h5", overwrite=True)

gc.enable()

In [0]:
!mv bestNet1D.h5 /content/drive/My\ Drive/ugroz/1D

# Построение моделей на основе машинного обучения

## RandomForestClassifier

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score
# Создадим классификатор
random_forest = RandomForestClassifier(n_estimators = 100, random_state = 50)

In [0]:
random_forest.fit(X_train, Y_train)

In [0]:
joblib.dump(random_forest, 'random.pkl')

In [0]:
r_pred = random_forest.predict(X_val)
predictions = [round(value) for value in r_pred]

accuracy = accuracy_score(Y_val, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [0]:
!ls

In [0]:
!mv random.pkl /content/drive/My\ Drive/ugroz/FOOREST

# XGBoost

In [0]:
import xgboost
from sklearn.metrics import accuracy_score

In [0]:
xgb = xgboost.XGBClassifier()


In [0]:
xgb.fit(X_train, Y_train)

In [0]:
xgb.save_model('xgb.model')

In [0]:
mv xgb.model /content/drive/My\ Drive/ugroz/XBOOST

In [0]:
y_pred = xgb.predict(X_val)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(Y_val, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# LGB

In [0]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
gc.enable()

In [0]:
# CV Folds
seed = 42
folds = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)

In [0]:
lgbm = lgb.LGBMClassifier(
        objective = 'binary',
        boosting_type = 'gbdt',
        n_estimators = 3500,
        learning_rate = 0.05, 
        num_leaves = 250,
        min_data_in_leaf = 125, 
        bagging_fraction = 0.901,
        max_depth = 13, 
        reg_alpha = 2.5,
        reg_lambda = 2.5,
        min_split_gain = 0.0001,
        min_child_weight = 25,
        feature_fraction = 0.5, 
        silent = -1,
        verbose = -1,
        n_jobs = -1) 


In [0]:
   history_lgb = lgbm.fit(X_train, Y_train, 
                         eval_set=[(X_train, Y_train), (X_val, Y_val)], 
                         eval_metric = 'auc', 
                         verbose = 250, 
                         early_stopping_rounds = 100)

In [0]:
history_lgb.booster_.save_model('lgb.txt')

In [0]:
!ls

In [0]:
!mv lgb.txt /content/drive/My\ Drive/ugroz/LGM

## Подгружаем данные на случай сбоя при обработке данных

In [0]:
X_train = pd.read_csv('/content/drive/My Drive/ugroz/train_dan/X_train4.csv')
X_train.shape

In [0]:
X_val = pd.read_csv('/content/drive/My Drive/ugroz/train_dan/X_val4.csv')
X_val.shape

In [0]:
Y_val = pd.read_csv('/content/drive/My Drive/ugroz/train_dan/Y_val4.csv')
Y_val.shape

In [0]:
Y_train = pd.read_csv('/content/drive/My Drive/ugroz/train_dan/Y_train4.csv')
Y_train.shape

## Визулизируем важные признаки

In [0]:
x = df_train[cols]
y = df_train['HasDetections']

In [0]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt

mo = ExtraTreesClassifier()
mo.fit(x, y)
print(mo.feature_importances_) 
feat_importances = pd.Series(mo.feature_importances_, index=x.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()